# Quantum Computing with Q# Tutorial
Welcome to the iPython Notebook for the Quantum Computing tutorial for [this]() Medium article.          
Let's first start by looking at Q#, the language used to code Quantum Applications in the Quantum Development Kit. Don't worry, it's not too hard!                     

<small>If you aren't familiar with IPy Notebooks, just press `Shift + Enter` to move to the next cell.</small>

In [115]:
// Load some dependencies
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;

// Helper functions
// An operation to run an Operation n times and print how many zeros and ones
operation RunNTimes(op : Unit => Result, n : Int): Unit {
    mutable zeros = 0;
    for i in 1..n {
        let result = op();
        if IsResultZero(result){ 
            set zeros += 1;
        }
    }
    Message($"Zeros: {zeros} | Ones: {n - zeros}");
}

RunNTimes

## Q# Fundamentals
All functions in Q# are called `operation`. The format for Q# operations is ~ 
```q#
operation <name>(<input_args>): <return_dtype> { <body> }
```
> Note that the `return_dtype` for functions without a return value is `Unit`, which is the equivalent of `void` in C# or C/C++.

Below is the code for a basic Hello World operation, using the [`Message`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.message) function from Q#, which is the rough equivalent of `print` in Python, `Console.Write` in C# and `std::cout` in C++.

In [116]:
// Function to print "Hello World"
operation HelloWorld(): Unit {
    Message("Hello World from Q#");
}

HelloWorld

As Q# is a language that runs on Quantum Computers, executing operations works a bit differently. <br />Rather than direct execution, we run an IPYNB magic command - [`%simulate`](https://docs.microsoft.com/en-us/qsharp/api/iqsharp-magic/simulate) followed by the name of the operation for simulation.

In [117]:
%simulate HelloWorld

Hello World from Q#


()

Q# also supports the basic fundamental datatypes, such as `Int`, `Float` et cetera.                     
Below is a basic Addition operation that works on integers.   <br />                   
<small>Notice that the `return` keyword, present in languages like Python, C# and C/C++ is also present in Q#.</small>

In [118]:
// Function to add two integers in Q#
operation Add(x : Int, y: Int): Int {
    return x + y;
}

Add

To simulate our operation, we need to input the arguments in `key=value` format after the name of the operation, as shown below.

In [119]:
%simulate Add x=3 y=5

8

Cool! Now try to write a function `pow`, which takes two `Int` arguments, say `x` and `y`, and returns `x^y`           

<blockquote><details><summary>> <i>Click here for Hint</i></summary>The function <A href="https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.math.powi" target="_blank"><code>Microsoft.Quantum.Math.PowI</code></A> might help you out here</details></blockquote>

In [120]:
// DIY~1
operation Pow(x: Int, y: Int): Int{
    // TODO: Create a function that raises x to the power of y.
    return -1;
}

Pow

In [121]:
// An operation to check whether the Pow function is giving the right output
operation AssertPow(): Unit {
    if Pow(4,5) != 1024 {
        Message("Try again!");
    } else {
        if Pow(-4,5) != -1024 {
            Message("Try again! Hint: Check for negative bases!");
        } else {
            Message("Success!");
        }
    }
}

AssertPow

In [122]:
%simulate AssertPow

Try again!


()

Already forgot the syntax? No problem!             <br />                               
Find documentation for any object using `<object_name>?` in the cell, like below.

In [123]:
Microsoft.Quantum.Math.PowI?

Microsoft.Quantum.Math.PowI

### Quantum Measurement
As mentioned in the article, a Quantum bit (better known as a Qubit) is the fundamental unit of storage for quantum computers.<br />Let's write a program that allocates a qubit and returns its measurement.<br />
The function that returns the measurement of a qubit in Q# is `M(Qubit) => Result`

In [124]:
operation CreateAndReturnMeasurement() : Result {
    // Allocate the qubit.
    use q = Qubit();
    // Measures the qubit.
    let result = M(q);
    return result;
}
operation nCreateAndReturnMeasurement(n: Int) : Unit {
    RunNTimes(CreateAndReturnMeasurement,n);
}

CreateAndReturnMeasurement, nCreateAndReturnMeasurement

In [125]:
%simulate CreateAndReturnMeasurement 

Zero

Great! The output indicates that this Quantum bit is in its initial state.<br /> Let's run the same function, say, a hundred times to see if the values change...

In [126]:
%simulate nCreateAndReturnMeasurement n=100

Zeros: 100 | Ones: 0


()

As expected, this returns zeros a hundred percent of the times...

### Superposition
Superposition is the unique state of Quantum Particles, where their value is the combination of infinite probabilities. <br />
Recall that measuring a qubit in superposition collapses it to one of the two binary values. <br />
A special *Qubit gate* that puts a Quantum particle into Superposition, in Q#, is the H transformation `H(Qubit)=>Unit`.
<details><summary><i>> More to know</i></summary>The Hadamard (aforementioned H) transformation is one which sets the Qubit's base states in such a way that a Quantum measurement causing collapse has an equal probability of returning zero or one. To know the mathematical details of transformations and quantum particles, see Microsoft's documentation <A href="https://docs.microsoft.com/en-us/azure/quantum/overview-algebra-for-quantum-computing" target=_blank>here</A></details>
<br />
This means that we can make a true Random Number Generator (RNG) using Qubits!
The steps in this would be ~

* Allocate the Qubit
* Put the Qubit into Superposition
* Measure the Qubit and return its value.
> Note that Qubits, in order to be reusable, need to be reset after every measurement. For this, we use a combined operation of Measurement and Resetting, i.e., [`MResetZ`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.measurement.mresetz)

In [127]:
// Creates a Quantum Random Number Generator using Qubits
operation QRNG() : Result {
    // Allocate the qubit
    use q = Qubit();
    // Put the Qubit in superposition
    H(q);
    // Measure it and reset the Qubit
    return MResetZ(q);
}
// Run it n times
operation nQRNG(n : Int) : Unit {
    RunNTimes(QRNG, n);
}

QRNG, nQRNG

In [128]:
%simulate nQRNG n=100

Zeros: 48 | Ones: 52


()

Now let's try to extend this, to getting a number within a specified limit.                                 
As the below operation uses a lot of new functions and syntax, let's jump right into it!                      
The main process here is ~
* Creating an Array of `Result` Type. Set it to mutable, so that we can append values to it later.
* As `Result` is effectively just a bit, we can construct a new integer by putting `n` of these bits together. The function [`BitSizeI`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.math.bitsizei) helps us calculate `n`, i.e., the total number of bits required to represent an integer, in this case the value `max`.
* We iterate over from 1 to `n` and append the quantum random bit value to the result array as we go.
* After the iteration, we use [ResultArrayAsInt](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.convert.resultarrayasint) to convert this array of Bits to an integer.
* Note that the value `BitSizeI(max)` may be capable of holding values *greater* than `max`, and in such a case, we will need to retry the operation till we get a value under it. This recursion is the reason for the `Warning QS5002`, and it is perfectly safe to ignore. 
* If it *is* ≤ max, we can safely output the number. For this, I've used the `?|` ternary operator, which is great syntactical sugar for `<bool> ? <value if true> | <value if false>`. This is equivalent to the `?:` ternary operator in C# and C/C++. 

In [129]:
// A Quantum Random Number Generator which returns a random value in the range [0, max]
operation QRNGLimit(max : Int) : Int {
        // n is the Minimum bit size required to represent max
        let n = BitSizeI(max);
        // Create a bit array. Size of the array isn't of any importance as we are going to append values anyways.
        mutable bitArray = new Result[0];
        // Iterate from 1 through n
        for _ in 1..n {
            // Append the bitArray
            set bitArray += [QRNG()];
        }
        // Convert BitArray to Integer
        let sample = ResultArrayAsInt(bitArray);
        // If sample > max, then try 
        return sample > max
               ? QRNGLimit(max)
               | sample;
    }

D:\snippet_.qs(16,18): warning QS5002: This expression may be short-circuited, and operation calls may not be executed.


QRNGLimit

Let's simulate this function! Try running this multiple times, and with multiple `max` values to confirm that it returns truly random numbers!

In [130]:
%simulate QRNGLimit max=200

130

Great! As a final TODO task, try to extend `QRNGLimit` to an operation `QRNGRange` that outputs random numbers within a range!
<blockquote><details><summary><i>> Click for Hint!</i></summary><code>QRNGLimit</code> is effectively <code>QRNGRange</code> with <code>min = 0</code>. What are the adjustments we need to do to allow us to consider that?<blockquote><details><summary><i>> Need more help?</i></summary>If we run <code>QRNGLimit</code> with <code>max - min</code> as <code>max</code>, it allows us to later compensate by returning the sum of <code>min</code> and the random value</details></blockquote></details></blockquote>

In [131]:
// Returns a random number in the range [min, max]
operation QRNGRange(min: Int, max: Int): Int {
    // TODO: Use the operation QRNGLimit and extend it to a minimum and maximum value.
    return -1;
}

QRNGRange

In [132]:
%simulate QRNGRange min=25 max=28

-1

## Conclusion
We're done! The purpose of this tutorial has been to invoke interest in Quantum Computing, in a way that does not require any mathematical qualification, in addition to equipping the reader with tools to take their first step in Quantum Computing using Q# and the QDK. <br /><br />To continue in your journey towards expertise in Quantum Computing, I encourage you to check out the below resources from Microsoft's Documentation.<br /> <br />
**Tools for learning Math**
* [Linear Algebra for Quantum Computing](https://docs.microsoft.com/en-us/azure/quantum/overview-algebra-for-quantum-computing)
* [Vectors and Matrices](https://docs.microsoft.com/en-us/azure/quantum/concepts-vectors-and-matrices)
* [The Qubit](https://docs.microsoft.com/en-us/azure/quantum/concepts-the-qubit)
<br>

Additionally, the [Quantum Katas](https://github.com/microsoft/QuantumKatas) are an excellent way to start your journey in Q# and Quantum computing. 
<br />
<h5>I hope you enjoyed this tutorial, and have a great day ahead!</h5>
